# 라이브러리


In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 파일경로


In [ ]:
# 1. CSV 파일 불러오기
file_path = "/content/drive/MyDrive/소프개인/new.csv"  # Colab 경로 또는 적절한 파일 경로로 변경
data = pd.read_csv(file_path)

# 데이터 전처리


In [ ]:
timesteps = 6  # 예: 3초의 영상에서 0.5초 간격으로 추출된 경우
feature_size = data.shape[1] - 1  # Label 열을 제외한 나머지 데이터

# 입력 데이터와 라벨 분리
X = data.iloc[:, :-1].values  # 마지막 열인 'Label'을 제외한 나머지 데이터
y = data['Label'].values

# 라벨 인코딩 (문자열 라벨을 정수로 변환)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)



In [ ]:
X_seq = []
y_seq = []

# timesteps 길이만큼 데이터를 묶어서 시퀀스 생성
for i in range(len(X) - timesteps):
    X_seq.append(X[i:i+timesteps])  # X의 timesteps 묶음 추가
    y_seq.append(y[i + timesteps])  # 각 시퀀스 다음의 y 값을 라벨로 추가

# 리스트를 배열로 변환
X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# 최종 데이터 크기 확인
print("X_seq shape:", X_seq.shape)  # 예상 출력: (samples, timesteps, features)
print("y_seq shape:", y_seq.shape)  # 예상 출력: (samples,)


X_seq shape: (7536, 6, 111)
y_seq shape: (7536,)


In [ ]:
print(feature_size)

111


# 학습데이터 분할


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.25, random_state=42, stratify=y_seq)

# 파이토치 텐서변환


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 5. PyTorch Tensor로 변환 (NaN을 -1로 대체)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# NaN 값을 -1로 대체
X_train = torch.nan_to_num(X_train, nan=-1.0)
X_test = torch.nan_to_num(X_test, nan=-1.0)


In [ ]:
# 데이터 검증
print("NaN in X_train:", np.isnan(X_train).sum())
print("Inf in X_train:", np.isinf(X_train).sum())


NaN in X_train: tensor(0)
Inf in X_train: tensor(0)


# 모델정의

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # LSTM에 입력
        h_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        c_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)

        out, _ = self.lstm(x, (h_0, c_0))  # LSTM의 출력
        out = out[:, -1, :]  # 마지막 time step의 출력 사용
        out = self.fc(out)
        return out

# 하이퍼파라미터 설정
input_size = feature_size
hidden_size = 64
num_layers = 2
num_classes = len(np.unique(y))
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)

"""# 손실함수와 최적화"""

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
import torch
import torch.optim as optim

# 하이퍼파라미터 설정
epochs = 150
learning_rate = 0.001

# 옵티마이저 초기화 (에포크 밖에서)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()  # 모델을 학습 모드로 설정

    optimizer.zero_grad()  # 기울기 초기

    # 순전파
    outputs = model(X_train)  # Pass input data through the model
    loss = criterion(outputs, y_train)  # Compute loss

    # 역전파
    loss.backward()  # 기울기 계산
    optimizer.step()  # 파라미터 업데이트

    # 에포크마다 손실 출력
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [10/150], Loss: 2.9502
Epoch [20/150], Loss: 2.6972
Epoch [30/150], Loss: 2.2660
Epoch [40/150], Loss: 1.8117
Epoch [50/150], Loss: 1.4599
Epoch [60/150], Loss: 1.1850
Epoch [70/150], Loss: 0.9624
Epoch [80/150], Loss: 0.7835
Epoch [90/150], Loss: 0.6450
Epoch [100/150], Loss: 0.5317
Epoch [110/150], Loss: 0.4403
Epoch [120/150], Loss: 0.3684
Epoch [130/150], Loss: 0.3108
Epoch [140/150], Loss: 0.2638
Epoch [150/150], Loss: 0.2264


# 손실함수와 최적화

# 모델학습

# 모델평가

In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 90.34%


In [ ]:
# 모델 전체 저장
torch.save(model, '/90_newcsv.pth')


In [ ]:
import torch
from sklearn.metrics import classification_report

# 테스트 데이터셋 로드 (예시로 Tensor 사용)
# X_test와 y_test는 테스트 데이터와 라벨을 텐서로 준비한 것입니다
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

# 모델을 평가하는 함수 정의
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, preds = torch.max(outputs, 1)

    # classification report 출력
    report = classification_report(y_test.numpy(), preds.numpy())
    print(report)

# 사용 예시
evaluate_model(model, X_test, y_test)


<ipython-input-18-596f56c34bcc>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)


              precision    recall  f1-score   support

           0       0.92      0.99      0.95        88
           1       0.96      0.98      0.97        90
           2       0.85      0.74      0.79        90
           3       0.81      0.84      0.83        88
           4       0.84      0.88      0.86        88
           5       0.99      0.96      0.97        72
           6       0.91      0.90      0.90        88
           7       0.85      0.89      0.87        89
           8       0.87      0.97      0.91        87
           9       0.95      0.89      0.92        90
          10       0.97      0.97      0.97        72
          11       0.92      0.89      0.90        88
          12       0.81      0.81      0.81        89
          13       0.90      0.84      0.87        89
          14       0.98      0.92      0.95        90
          15       0.73      0.75      0.74        89
          16       0.97      1.00      0.99        72
          17       0.93    